In [1]:
# if else statement inside py_func causes problem beware!
# nopes it seems its working fine

In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
audio_to_mfcc('/home/saurabh/feeny_dataset/data/generated/feeny_62.wav').shape

(357, 13)

In [5]:
timesteps=300

In [6]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [7]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("/home/saurabh/feeny_dataset/data/words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i


In [8]:
print(len(word_dictionary))

52


In [9]:
def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
       
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list

In [10]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [11]:
def _read_py_function(audio, label):
    audio = audio_to_mfcc(audio)
    #label=word_to_index(label.decode("utf-8"))
    #temp=0
    if audio.shape[0] < timesteps:
        original_length=audio.shape[0]
        print(original_length)
        audio=pad(audio)
       # return audio ,label, original_length
        
    
    elif audio.shape[0] >= timesteps:
        original_length=timesteps
        audio=pad(audio)
       # return audio ,label, original_length
        
        
    return audio ,label, original_length
    
    


In [12]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/feeny.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype, tf.int64])))




dataset=dataset.batch(2)

In [32]:
num_lines = sum(1 for line in open('/home/saurabh/Documents/tf_orange/tf_orange/feeny.csv'))
print(num_lines)


if num_lines%2 != 0:
    print("that's odd!")
    dataset2=dataset.take((num_lines - 1 ))
    

9
that's odd!


In [33]:
iterator = dataset2.make_initializable_iterator()
next_element = iterator.get_next()

In [34]:
# Start training
with tf.Session() as sess:
   # sess.run(init)
    
    # op to write logs to Tensorboard
   # summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    

    # Training cycle
    for epoch in range(1):
     #   avg_cost = 0.
        
        
        sess.run(iterator.initializer)
        while True:
            try:
                Next_element=sess.run(next_element)
                print( Next_element[1] )
                
            except tf.errors.OutOfRangeError:
                break

229
[b'kindly as possible the show has turned'
 b"press junket there's some learning going"]
[b'into a circus and you three are driving'
 b'masses for the first time in history the']
204
[b'so Gutenberg invented the printing press'
 b'common man had access to the same']
242
[b'darling gentlemen might I interrupt your'
 b'information that used to be available']
242
[b'making the written word available to the']


In [31]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(10):
      value = sess.run(next_element)
      print(value[2])

229
[300 229]
[300 300]
204
[300 204]
242
[300 242]
242
[242]


OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext_1 = IteratorGetNext[output_shapes=[<unknown>, <unknown>, <unknown>], output_types=[DT_DOUBLE, DT_STRING, DT_INT64], _device="/job:localhost/replica:0/task:0/cpu:0"](Iterator_1)]]

Caused by op 'IteratorGetNext_1', defined at:
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/zmq/eventloop/iol